In [3]:
import numpy as np
import librosa
from scipy.io.wavfile import write
import pywt
import os

# DFT from scratch
def dft(x):
    N = len(x)
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * n * k / N)
    return np.dot(M, x)

# FFT from scratch, adjusted to handle complex inputs
def fft(x):
    N = len(x)
    if N <= 1: return x
    if not isinstance(x[0], complex):  # Ensure input is complex
        x = np.asarray(x, dtype=complex)
    even = fft(x[0::2])
    odd = fft(x[1::2])
    T = np.exp(-2j * np.pi * np.arange(N // 2) / N) * odd
    return np.concatenate([even + T, even - T])

# Ensure audio_data length is a power of two
def adjust_audio_length(audio_data):
    N = len(audio_data)
    N_adjusted = 2**int(np.log2(N))
    return audio_data[:N_adjusted]

# Compress audio data using DFT
def compress_audio(audio_data, sample_rate, chunk_size=1024, compression_ratio=0.9):
    audio_data = adjust_audio_length(audio_data)
    num_chunks = int(len(audio_data) / chunk_size)
    audio_chunks = np.array_split(audio_data, num_chunks)
    compressed_chunks = []
    for chunk in audio_chunks:
        spectrum = fft(chunk)
        num_bins_to_keep = int(len(spectrum) * (1 - compression_ratio))
        spectrum[num_bins_to_keep:] = np.zeros(len(spectrum) - num_bins_to_keep, dtype=complex)
        compressed_chunk = np.fft.ifft(spectrum)
        compressed_chunks.append(compressed_chunk.real)
    return np.concatenate(compressed_chunks)

# DWT Compression Function
def compress_audio_dwt(audio_data, sample_rate, compression_ratio=0.9, wavelet='db1'):
    coeffs = pywt.wavedec(audio_data, wavelet)
    threshold = np.quantile(np.abs(coeffs[-1]), 1 - compression_ratio)
    compressed_coeffs = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]
    compressed_audio = pywt.waverec(compressed_coeffs, wavelet)
    return compressed_audio[:len(audio_data)]

# Load the audio signal
audio_file = '/content/drive/MyDrive/audio_SIP_PROJECT/noncompressed'
audio_data, sample_rate = librosa.load(audio_file, sr=None, mono=True)

# Apply DFT compression
compressed_audio_dft = compress_audio(audio_data, sample_rate)
output_path_dft = '/content/drive/MyDrive/audio_SIP_PROJECT/compressed_audio_dft.wav'
write(output_path_dft, sample_rate, compressed_audio_dft.astype(np.int16))

# Apply DWT compression
compressed_audio_dwt = compress_audio_dwt(audio_data, sample_rate)
output_path_dwt = '/content/drive/MyDrive/audio_SIP_PROJECT/compressed_audio_dwt.wav'
write(output_path_dwt, sample_rate, compressed_audio_dwt.astype(np.int16))

# Calculate and print file sizes and compression percentages
original_size = os.path.getsize(audio_file)
compressed_sizes = {
    'DFT': os.path.getsize(output_path_dft),
    'DWT': os.path.getsize(output_path_dwt)
}

print("Original file size:", original_size, "bytes")
for method, compressed_size in compressed_sizes.items():
    compression_percentage = (1 - (compressed_size / original_size)) * 100
    print(f"{method} Compressed file size:", compressed_size, "bytes")
    print(f"{method} Compression percentage:", compression_percentage, "%")


Original file size: 7410316 bytes
DFT Compressed file size: 2097196 bytes
DFT Compression percentage: 71.69896668374196 %
DWT Compressed file size: 3705180 bytes
DWT Compression percentage: 49.999703116574246 %
